# Observables
## Definition
Observables provide a unified way to access a large quantity of figures resulting from various
computations on lattices. They may be used in parameter scans, matching, response matrices, plots…

Observables values are usually scalars or numpy arrays of any shape, but may have any type.

An Observable has a {py:attr}`~.Observable.name`, optional {py:attr}`~.Observable.target`, {py:attr}`~.Observable.weight` and {py:attr}`~.Observable.bounds` attributes for matching,
a {py:attr}`~.Observable.label` for plot legends. After evaluation, it has the following main properties:
- {py:attr}`~.Observable.value`
- {py:attr}`~.Observable.weighted_value`: `value / weight`
- {py:attr}`~.Observable.deviation`:  `value - target`
- {py:attr}`~.Observable.weighted_deviation`:  `(value - target)/weight`
- {py:attr}`~.Observable.residual`:  `((value - target)/weight)**2`

For evaluation, observables must be grouped in an {py:class}`.ObservableList` which combines the needs of all its members, avoiding redundant computations. {py:class}`.ObservableList` provides the {py:meth}`~.ObservableList.evaluate` method, and the
{py:attr}`~.ObservableList.values`, {py:attr}`~.ObservableList.deviations`,
{py:attr}`~.ObservableList.residuals` and {py:attr}`~.ObservableList.sum_residuals` properties, among others.

```{attention}

Evaluation must only be done on ObservableLists, not directly on Observables. Evaluating directly an
Observable will miss the data computed at the list level to avoid redundant computations.

Only instances of the base {py:class}`.Observable`class may be directly evaluated since they don't
require any data
```

Observable values may depend on parameters being varied for different evaluations: for instance, optics results depend on the off-momentum *dp*, trajectory coordinates depend on the input coordinates *r_in*. These are called "evaluation parameters". They can be defined, in increasing priority
- at Observable instantation, to provide default values,
- in the containing ObservableList, to give common parameters to all its members,
- as arguments of the  {py:meth}`~.ObservableList.evaluate` method.

For optics specific Observables, the available evaluation parameters are documented with
the Observable. For user-defined Observables, any evaluation parameter can be defined.

```{attention}

For optics-specific Observables, all optics computations are made at the ObservableList
level for efficiency. So evaluation parameters defined at the Observable level will be ignored.

For instance the *ring*, *dp* parameters will be used in optics computations common to
all the Observables. Such parameters defined in single Observables are ignored
```

AT provides a number of specific observables sharing a common interface, inherited from the
{py:class}`.Observable` base class. They are:
- {py:class}`.RingObservable`: Any user-defined property depending upon *ring*
- {py:class}`.ElementObservable`: Any user-defined property at a specific location along the lattice
- {py:class}`.OrbitObservable`: Closed orbit: {math}`x_{co}`…,
- {py:obj}`.GlobalOpticsObservable`: global output of {py:meth}`~.Lattice.get_optics`: tunes, damping times…,
- {py:class}`.LocalOpticsObservable`: local output of {py:meth}`~.Lattice.get_optics`: {math}`\beta`, {math}`\eta`…,
- {py:class}`.MatrixObservable`: transfer matrix elements {math}`T_{ij}`…,
- {py:class}`.TrajectoryObservable`: {math}`x, p_x`…,
- {py:class}`.EmittanceObservable`: {math}`\epsilon_x`, {math}`\tau_x`…,
- {py:class}`.LatticeObservable`: attributes of lattice elements,
- {py:class}`.GeometryObservable`: floor positions of elements,
- {py:class}`.LocalEmittanceObservable`: local output of {py:meth}`~.Lattice.ohmi_envelope`

Custom Observables may be created by providing the adequate evaluation function.

## Observable gallery
These examples show how to declare various Observables, how to evaluate them and how to extract
and display their values.
### Setup the environment

In [1]:
import at
import numpy as np
from importlib.resources import files, as_file
import time

In [2]:
from at import (
    Observable,
    ObservableList,
    RingObservable,
    OrbitObservable,
    GlobalOpticsObservable,
    LocalOpticsObservable,
    MatrixObservable,
    TrajectoryObservable,
    EmittanceObservable,
    LatticeObservable,
    GeometryObservable,
    Need,
)

### Load a test lattice

In [3]:
fname = "hmba.mat"
with as_file(files("machine_data") / fname) as path:
    hmba_lattice = at.load_lattice(path)

### Create Observables

Create an empty ObservableList:

In [4]:
obs1 = ObservableList()

Horizontal closed orbit on all Monitors:

In [5]:
obs1.append(OrbitObservable(at.Monitor, axis="x"))

Create a 2{sup}`nd` ObservableList:

In [6]:
obs2 = ObservableList()

Vertical $\beta$ at all monitors, with a target and bounds.

The vertical $\beta$ is constrained in the interval
[*target*+*low_bound* *target*+*up_bound*], so here [*-Infinity 7.0*]

The residual will be zero within the interval.

In [7]:
obs2.append(
    LocalOpticsObservable(
        at.Monitor, "beta", plane=1, target=7.0, bounds=(-np.inf, 0.0)
    )
)

check the concatenation of ObservableLists:

In [8]:
allobs = obs1 + obs2

Full transfer matrix to `BPM02`:

In [9]:
allobs.append(MatrixObservable("BPM_02"))

Maximum of vertical beta on monitors:

In [10]:
allobs.append(LocalOpticsObservable(at.Monitor, "beta", plane="v", statfun=np.amax))

First 4 coordinates of the closed orbit at Quadrupoles:

In [11]:
allobs.append(
    LocalOpticsObservable(
        at.Quadrupole, "closed_orbit", plane=slice(4), target=0.0, weight=1.0e-6
    )
)

Position along the lattice of all quadrupoles:

In [12]:
allobs.append(LocalOpticsObservable(at.Quadrupole, "s_pos"))

Horizontal tune with the integer part:

In [13]:
allobs.append(GlobalOpticsObservable("tune", plane=0, use_integer=True))

Total phase advance at the end of the lattice (all planes):

In [14]:
allobs.append(LocalOpticsObservable(at.End, "mu"))

Horizontal W-function:

In [15]:
allobs.append(LocalOpticsObservable(at.Sextupole, "W", plane="x"))

Chromaticity in all planes:

In [16]:
allobs.append(GlobalOpticsObservable("chromaticity"))

Average of sextupole strengths:

In [17]:
allobs.append(LatticeObservable(at.Sextupole, "H", statfun=np.mean))

Strengths of all sextupoles:

In [18]:
allobs.append(LatticeObservable(at.Sextupole, "PolynomB", index=2))

Horizontal emittance:

In [19]:
allobs.append(EmittanceObservable("emittances", plane="x"))

p{sub}`x` component of the trajectory on all monitors:

In [20]:
allobs.append(TrajectoryObservable(at.Monitor, axis="px"))

Floor position of all monitors:

In [21]:
allobs.append(GeometryObservable(at.Monitor, "x"))

Horizontal beam size {math}`\sigma_x} at monitor positions:

In [22]:
allobs.append(at.LocalEmittanceObservable(at.Monitor, "sigma6", axis=0))

### Evaluation

An input trajectory is required for the trajectory Observable

In [23]:
r_in = np.zeros(6)
r_in[0] = 0.001
r_in[2] = 0.001
values = allobs.evaluate(
    ring=hmba_lattice.enable_6d(copy=True), r_in=r_in, dp=0.0, initial=True
)

### Extract a single Observable value
index 7: total phase advance

In [24]:
allobs[7].value

array([[1.49638018e+01, 5.36820522e+00, 6.85246956e-04]])

### Get the list of all Observable values:

In [25]:
allobs.values

(array([-3.02197151e-09,  4.50706098e-07,  4.08215750e-07,  2.37905632e-08,
        -1.31787024e-08,  2.47236652e-08, -2.95318222e-08, -4.05608194e-07,
        -4.47409214e-07, -2.24856454e-09]),
 array([5.30279703, 7.17604152, 6.55087808, 2.31448878, 3.40498444,
        3.405044  , 2.3146451 , 6.55106241, 7.17614175, 5.30283837]),
 array([[[-1.08194106e+00,  3.18809568e+00,  0.00000000e+00,
           0.00000000e+00,  8.22407787e-02, -1.72158067e-05],
         [-6.80522735e-01,  1.08099571e+00,  0.00000000e+00,
           0.00000000e+00,  4.90131193e-02, -1.02601216e-05],
         [ 0.00000000e+00,  0.00000000e+00,  7.55929650e-01,
           3.87059271e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.79279293e-01,
          -2.15524755e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.13313809e-08, -1.08618621e-07,  0.00000000e+00,
           0.00000000e+00,  9.99995907e-01, -2.09333332e-04],
         [ 2.93742345e-03,  6.73567965e-02,  0.0000

### Get a pretty output of all Observables.

As no variation was made, *Actual* values are always equal to *Initial* values.

The deviation is zero for all Observables for which no *target* was specified

In [26]:
print(allobs)


    location              Initial            Actual         Low bound        High bound         deviation 
orbit[x]
    BPM_01           -3.02197e-09      -3.02197e-09               -                 -                 0.0 
    BPM_02            4.50706e-07       4.50706e-07               -                 -                 0.0 
    BPM_03            4.08216e-07       4.08216e-07               -                 -                 0.0 
    BPM_04            2.37906e-08       2.37906e-08               -                 -                 0.0 
    BPM_05           -1.31787e-08      -1.31787e-08               -                 -                 0.0 
    BPM_06            2.47237e-08       2.47237e-08               -                 -                 0.0 
    BPM_07           -2.95318e-08      -2.95318e-08               -                 -                 0.0 
    BPM_08           -4.05608e-07      -4.05608e-07               -                 -                 0.0 
    BPM_09           -4.474

## User-defined evaluation functions

User-defined functions receive as keyword arguments all the evaluation parameters specified
on their Observable instantiation, on the containing ObservableList and as arguments of the
{py:meth}`~.ObservableList.evaluate` method. They must therefore be ready to accept and ignore
all keywords targeting other Observables.

### With the base {py:class}`.Observable` class

When using the base {py:class}`.Observable` class, the evaluation function is entirely responsible to provide the return value. It is called as `value = fun(*eval_args, **eval_kw)`. *eval_args* is a tuple of
positional evaluation arguments defined at the Observable instantiation, *eval_kw* is the dictionary of evaluation keywords.

The evaluation function returns the current date and time. We define the output format as an
evaluation parameter named `format`:

In [27]:
def now(format="%j", **_):
    # ignore all other keywords
    return time.strftime(format)

We can now create two observables using this function, the second forcing the date format to `%c`

In [28]:
allobs = ObservableList([Observable(now), Observable(now, format="%c")])

Evaluation returns both different results:

In [29]:
allobs.evaluate()
for obs in allobs:
    print(f"{obs.name!r}: {obs.value}")

'now': 342
'now': Mon Dec  8 11:42:08 2025


But if we give a `format` to {py:meth}`~.ObservableList.evaluate`, it will have priority in both cases:

In [30]:
allobs.evaluate(format="%A")
for obs in allobs:
    print(f"{obs.name!r}: {obs.value}")

'now': Monday
'now': Monday


### With {py:class}`.RingObservable`

The evaluation function receives the lattice attached to the {py:class}`.RingObservable`.
It is called as `value = fun(ring, **eval_kw)`. The lattice is given as a positional
argument. All the evaluation keywords are also provided and can be ignored if irrelevant.
By default the observable name is taken as the name of the evaluation function.

In [31]:
allobs = ObservableList()

**Lattice circumference**:

In [32]:
def circumference(ring, **_):
    # ignore all other keywords
    return ring.circumference

allobs.append(RingObservable(circumference))

**Lattice momentum compaction**:

As {py:meth}`~.Lattice.get_mcf` needs a 4D lattice, we need to make sure
that the given lattice is 4D, whatever the lattice given to {py:meth}`~.ObservableList.evaluate`:

In [33]:
def momentum_compaction(ring, dp=0.0, **_):
    # ignore all other keywords
    return ring.get_mcf(dp=dp)

allobs.append(RingObservable(momentum_compaction, needs=Need.NEED_4D))

**Evaluation**:

In [34]:
allobs.evaluate(ring=hmba_lattice.enable_6d(copy=True), dp=0.01)
for obs in allobs:
    print(f"{obs.name!r}: {obs.value}")

'circumference': 843.9772144741422
'momentum_compaction': 8.820785280500313e-05


### With {py:class}`.LocalOpticsObservable`

The evaluation function receives the optical data computed at the specified locations.
It is called as `value = fun(elemdata, **eval_kw)` where *elemdata* is the output of {py:meth}`~.Lattice.get_optics` for all the locations specified in *refpts*.

The return value must have as many lines as observation points (the length of *elemdata*) and any number of columns.
The column may be selected in the output with the *plane* keyword.

In [35]:
allobs = ObservableList()

**Phase advance between 2 points**:

We ask for the phase at 2 points and take the difference

In [36]:
def phase_advance(elemdata, **_):
    mu = elemdata.mu
    return mu[-1] - mu[0]

We need to set *all_points* to avoid jumps in the phase advance, and to set *summary* to tell that the evaluation
returns a single output instead of one output per observation point. We look at positions 33 and 101 and select
vertical plane:

In [37]:
allobs.append(
    LocalOpticsObservable(
        [33, 101], phase_advance, plane="y", all_points=True, summary=True
    )
)

**Beam size**:

We can compute the beam envelope for arbitrary emittances $\epsilon_{x|y}$ and momentum spread $\sigma_\delta$ using $\sigma_{x|y} = \sqrt{\beta_{x|y} \epsilon_{x|y} + (\eta_{x|y} \sigma_\delta)^2}$.

We will define the emittances as a *emit* evaluation keyword and the momentum spread as a *sigma_e* keyword:

In [38]:
def beam_size(elemdata, emit=None, sigma_e=None, **_):
    return np.sqrt(
        elemdata.beta * emit + (elemdata.dispersion[:, [0, 2]] * sigma_e) ** 2
    )

We set default values for the emittances at instantiation of the observable and look at all the monitors:

In [39]:
allobs.append(
    LocalOpticsObservable(
        at.Monitor, beam_size, emit=[130.0e-12, 10.0e-12], sigma_e=0.9e-3
    )
)

We can now evaluate the results for default options:

In [40]:
allobs.evaluate(ring=hmba_lattice)
for obs in allobs:
    print(f"{obs.name!r}: {obs.value}")

'phase_advance[y]': 2.9974197440054082
'beam_size': [[3.21226173e-05 7.28205547e-06]
 [8.04615146e-05 8.47119400e-06]
 [7.32761035e-05 8.09381727e-06]
 [1.95950639e-05 4.81099629e-06]
 [1.82841176e-05 5.83525760e-06]
 [1.82841255e-05 5.83526128e-06]
 [1.95950796e-05 4.81100216e-06]
 [7.32760882e-05 8.09380363e-06]
 [8.04614926e-05 8.47117301e-06]
 [3.21226186e-05 7.28203334e-06]]


But we can also evaluate with different parameters, for instance at 1% off-momentum and with different emittances:

In [41]:
allobs.evaluate(ring=hmba_lattice, dp=0.01, emit=[140.0e-12, 20.0e-12])
for obs in allobs:
    print(f"{obs.name!r}: {obs.value}")

'phase_advance[y]': 2.944614942817739
'beam_size': [[3.56856110e-05 1.02701261e-05]
 [7.99928036e-05 1.22514050e-05]
 [7.27788329e-05 1.17851226e-05]
 [1.89108125e-05 6.92431365e-06]
 [1.93833205e-05 8.05116711e-06]
 [1.93833291e-05 8.05117223e-06]
 [1.89108266e-05 6.92432199e-06]
 [7.27788189e-05 1.17851048e-05]
 [7.99927834e-05 1.22513769e-05]
 [3.56856115e-05 1.02700955e-05]]
